# Дипломная работа

Цель работы — разработка и экспериментальная оценка системы быстрого поиска в больших текстовых массивах с реализацией полнотекстового, семантического и гибридного поиска, позволяющей совмещать скорость и качество релевантности.

Тестирование и оценка методов полнотекстового, семантического и гибридного поиска проводились на корпусе данных, собранном из новостного агрегатора Лента.ру.

Объём корпуса: около 800 000 строк текстовых данных, включающих заголовки и полные тексты новостей.

Предобработка текста: токенизация, удаление стоп-слов, лемматизации.

Цель эксперимента: оценить эффективность и производительность каждого метода поиска на крупном реальном корпусе текстов.

Метрики оценки:

Precision — точность найденных релевантных документов,

Recall — полнота поиска,

F1-score — гармоническое среднее Precision и Recall,

Latency — среднее время выполнения запроса


In [ ]:
# Установка библиотек:
# pip install elasticsearch sentence-transformers faiss-cpu pandas tqdm nltk pymorphy3

import os
import json
import time
import string
import pandas as pd
import numpy as np
import requests
import pymorphy3
import nltk
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.metrics import precision_score, recall_score, f1_score
import subprocess

# Загрузка необходимых ресурсов NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

#Запуск Elasticsearch
bat_file = r"C:\Users\zoira\PycharmProjects\scientificProject4\elasticsearch-8.17.3\bin\elasticsearch.bat"
subprocess.Popen([
    "powershell",
    "Start-Process", "cmd",
    f'-ArgumentList "/c {bat_file}"',
    "-Verb", "RunAs"
])

url = "http://localhost:9200"
username = "elastic"
password = "Vje+ivqy8byb0-oU=Emr"

max_tries = 15
for i in range(max_tries):
    try:
        response = requests.get(url, auth=(username, password))
        if response.status_code == 200:
            print("Elasticsearch запущен и доступен.")
            break
    except requests.exceptions.ConnectionError:
        pass
    print(f"Ожидание запуска Elasticsearch... попытка {i+1}/{max_tries}")
    time.sleep(2)
else:
    raise RuntimeError("Не удалось подключиться к Elasticsearch. Проверьте запуск сервера")

#Загрузка данных
csv_file = r'C:\Users\zoira\Downloads\processed_news.csv'
chunks = []
chunk_size = 10000
print("Загрузка CSV файла...")
for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    chunks.append(chunk)
df = pd.concat(chunks, ignore_index=True)
print(f"Загружено строк: {len(df)}")

#Подключение к Elasticsearch
es = Elasticsearch("http://localhost:9200", basic_auth=(username, password))
if not es.ping():
    raise RuntimeError("Elasticsearch недоступен после запуска!")

#Предобработка текста
stop_words = set(stopwords.words("russian"))
punctuation_marks = set(string.punctuation)
morph = pymorphy3.MorphAnalyzer()

def preprocess(text, stop_words, punctuation_marks, morph):
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text.lower())
    lemmas = []
    for token in tokens:
        if token in punctuation_marks:
            continue
        p = morph.parse(token)
        if not p:
            continue
        lemma = p[0].normal_form
        if lemma in stop_words:
            continue
        lemmas.append(lemma)
    return lemmas

df["clean_content"] = df["title"].fillna('') + " " + df["text"].fillna('')
df["clean_tokens"] = df["clean_content"].apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))

#Подготовка документов для индексации
docs = [{
    "id": str(i),
    "title": row["title"] if isinstance(row["title"], str) else "",
    "text": row["text"] if isinstance(row["text"], str) else "",
    "content": " ".join(row["clean_tokens"])
} for i, row in df.iterrows()]

INDEX = "news-eval"

#Пересоздание индекса
if es.indices.exists(index=INDEX):
    es.indices.delete(index=INDEX)
es.indices.create(index=INDEX, ignore=400)

#Bulk индексация
actions = ({
    "_index": INDEX,
    "_id": doc["id"],
    "_source": doc
} for doc in docs)
helpers.bulk(es, actions)


#Семантический поиск через BERT и FAISS
model = SentenceTransformer("DeepPavlov/rubert-base-cased")

#Генерация эмбеддингов
texts = [doc["content"] for doc in docs]
embs = model.encode(texts, convert_to_numpy=True)

dimension = embs.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embs)
id_map = {i: doc["id"] for i, doc in enumerate(docs)}

#Метки релевантности для оценки
true_relevance = {
    "образование": {"0", "1", "4"},
    "экономика": {"2", "5"},
    "спорт": {"3", "6"}
}

def evaluate(query, top_k=5):
    true_ids = true_relevance.get(query, set())

    # Полнотекстовый поиск
    t1 = time.time()
    es_resp = es.search(index=INDEX, query={"match": {"content": query}}, size=top_k)
    t2 = time.time()
    ft_ids = {hit["_id"] for hit in es_resp["hits"]["hits"]}

    # Семантический поиск
    q_emb = model.encode([query], convert_to_numpy=True)
    t3 = time.time()
    _, I = faiss_index.search(q_emb, top_k)
    t4 = time.time()
    sem_ids = {id_map[i] for i in I[0]}

    # Гибридный поиск
    hybrid_ids = ft_ids & sem_ids
    t5 = time.time()

    docs_ids = [doc["id"] for doc in docs]

    def compute_metrics(pred_ids, true_ids):
        y_true = [doc_id in true_ids for doc_id in docs_ids]
        y_pred = [doc_id in pred_ids for doc_id in docs_ids]
        return {
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "f1": f1_score(y_true, y_pred, zero_division=0)
        }

    metrics = {
        "Fulltext": compute_metrics(ft_ids, true_ids),
        "Semantic": compute_metrics(sem_ids, true_ids),
        "Hybrid": compute_metrics(hybrid_ids, true_ids)
    }

    latency = {
        "Fulltext": (t2 - t1) * 1000,
        "Semantic": (t4 - t3) * 1000,
        "Hybrid": (t5 - t3) * 1000
    }

    return metrics, latency

#Пример оценки
query = "образование"
metrics, latency = evaluate(query)

print(f"\nСравнение методов по запросу: '{query}'")
print("Метод       | Precision | Recall | F1   | Latency (ms)")
for method in metrics:
    m = metrics[method]
    l = latency[method]
    print(f"{method:<12} | {m['precision']:.2f}     | {m['recall']:.2f}   | {m['f1']:.2f} | {l:.1f}")
